In [3]:
import torch
import numpy as np
import pandas as pd

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [6]:
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=658)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
#from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#model_name = "sberbank-ai/sbert_large_nlu_ru"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=658)
#попробуем позже

In [7]:
torch.cuda.empty_cache()
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
df = pd.read_csv("../../draft.csv")
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df['id'] = pd.Series(df['id']).astype(str)

In [ ]:
g_id = []
for id in df['id']:
    if len(id) == 10:
        g_id.append(str(id)[:4])
    else:
        g_id.append('0' + str(id)[:3])
df['g_id'] = g_id

In [ ]:
counts = df.value_counts('id')
df_count = pd.DataFrame(data={'id': counts.index, 'count': counts.values})

In [ ]:
df_count.to_csv('../../tnveds_sort.csv')

In [ ]:
df['label'] = df['label'].apply(lambda x: ' '.join(x.split()[:400]) if len(x.split())>400 else x)

In [ ]:
a = df['label'].apply(lambda x: len(x.split())).hist(figsize=(15, 10))

In [ ]:
df['label'][df['label'].apply(lambda x: True if len(x.split()) == 1 else False)]

In [ ]:
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.fit(df['g_id'])

In [ ]:
df['g_id'] = Label_encoder.fit_transform(df['g_id'])

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df.columns = ['description', 'labels']

In [ ]:
df.to_csv('../../label_enc.csv', index=False)

In [ ]:
np.save('../../classes.npy', Label_encoder.classes_)

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:int(len(df)*0.8)]
test =  df[int(len(df)*0.8):]

In [ ]:
print(len(test['labels'].value_counts()))

In [ ]:
train.to_csv('../../train.csv', index=False)
test.to_csv('../../test.csv', index=False)

In [ ]:
model.named_parameters

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
!pip install datasets

In [8]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'train': '../../train.csv', 'test': '../../test.csv'})

Found cached dataset csv (C:/Users/yeril/.cache/huggingface/datasets/csv/default-c2d1908b4f31630f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 333.34it/s]


In [9]:
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)

In [13]:
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))

In [14]:
set(dataset['train'].features.keys())


{'attention_mask', 'input_ids', 'labels', 'token_type_ids'}

In [15]:
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 

In [16]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=1)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=1)
#если есть видеопамять около 10гб, можно поставить 8

In [ ]:
len(train_dataloader)

In [17]:

optimizer = AdamW(model.parameters(), lr=2e-6)

num_epochs = 4

num_training_steps = num_epochs * len(train_dataloader)


lr_scheduler = get_scheduler(
    "constant_with_warmup",
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [21]:
from tqdm import tqdm

In [ ]:
next(iter(train_dataloader))

In [ ]:
best_f1 = 0.
show_train_loss_every_num_epoch = 0.1

for epoch in range(num_epochs):
    print(40*'-', '\nepoch', epoch+1)
    model.train()
    losses = []

    for i, batch in enumerate(train_dataloader):
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        losses.append(loss.item())

        if i%int(len(train_dataloader)*show_train_loss_every_num_epoch)==int(len(train_dataloader)*show_train_loss_every_num_epoch)-1:
            print(f'train loss [{i*100/len(train_dataloader):.2f}%]: {np.array(losses).mean():.3f}')
            losses = []
            print('\nvalidating')

            f1 = load_metric('f1')
            acc = load_metric('accuracy')
            precision = load_metric('precision')
            recall = load_metric('recall')
            with torch.no_grad():
                model.eval()
                
                for batch in tqdm(test_dataloader):
                    batch = {k: v.to(device) for k, v in batch.items()}
                    with torch.no_grad():
                        outputs = model(**batch)
                    logits = outputs.logits
                    predictions = torch.argmax(logits, dim=-1)
                    f1.add_batch(predictions=predictions, references=batch["labels"])    
                    acc.add_batch(predictions=predictions, references=batch["labels"])    
                    precision.add_batch(predictions=predictions, references=batch["labels"])    
                    recall.add_batch(predictions=predictions, references=batch["labels"])    

                print('weighted summary:')
                print('Test acc:', acc.compute()['accuracy'])
                print('Test precision:', precision.compute(average = 'weighted')['precision'])
                print('Test recall:', recall.compute(average = 'weighted')['recall'])
                f1_weighted = f1.compute(average = 'weighted')['f1']
                print('Test f1:', f1_weighted, '\n')

                if f1_weighted > best_f1:
                    best_f1 = f1_weighted
                    model.save_pretrained(f"best_model_nov_2021_f1_max={best_f1}_len=100")

    model.train()

---------------------------------------- 
epoch 1
train loss [10.00%]: 3.701

validating


100%|████████████████████████████████████████████████████████████████████████████| 13992/13992 [11:42<00:00, 19.93it/s]


weighted summary:
Test acc: 0.36556603773584906


C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test precision: 0.2805728291815288
Test recall: 0.36556603773584906
Test f1: 0.28694679431793496 

train loss [20.00%]: 3.397

validating


...

weighted summary:
Test acc: 0.5417381360777587


C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test precision: 0.43189806248974055


C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test recall: 0.5417381360777587
Test f1: 0.4539413574939823 



In [ ]:
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))